# Replace Active Cycle with Latest Template

This notebook replaces the current active cycle with a fresh copy from the latest template, **preserving the same cycle name**.

**Use this when you need to:**
- Reset a cycle to start fresh with clean notebooks
- Update an existing cycle with new template changes
- Fix a corrupted cycle directory

**⚠️ WARNING:**
- This will **PERMANENTLY DELETE** the current active cycle directory
- This will **PERMANENTLY DELETE** all database records (stages, steps, runs, batches, jobs, configurations)
- **This action cannot be undone!**
- No archive copy will be created

**Process:**
1. Delete the current active cycle directory
2. Delete the cycle from the database (CASCADE deletes all related data)
3. Create a new cycle with the same name using the latest template
4. Register stages and steps from the template

In [ ]:
%load_ext autoreload
%autoreload 2

import shutil
from pathlib import Path
from helpers import cycle, ux, constants
from helpers.context import WorkContext

## Check Current Active Cycle

In [ ]:
ux.header("Current Active Cycle")

# Get active cycle
active_cycle = cycle.get_active_cycle()

if not active_cycle:
    ux.error("No active cycle found. Use 'New Cycle' notebook to create one.")
    raise Exception("No active cycle to replace")

cycle_name = active_cycle['cycle_name']
ux.success(f"Active cycle: {cycle_name}")
ux.info(f"Created: {active_cycle['created_ts']}")

# Show cycle progress
progress_df = cycle.get_cycle_progress(cycle_name)
if not progress_df.empty:
    ux.subheader(f"Current Progress ({len(progress_df)} steps)")
    ux.step_progress(progress_df)
else:
    ux.info("No step execution history")

## Confirm Replacement

In [ ]:
ux.header("⚠️ CONFIRMATION REQUIRED")

ux.warning(f"You are about to REPLACE cycle: {cycle_name}")
print()
print("This will:")
print(f"  1. PERMANENTLY DELETE the Active_{cycle_name} directory")
print(f"  2. PERMANENTLY DELETE all database records (stages, steps, runs, batches, jobs, configurations)")
print(f"  3. Create a fresh copy from the latest template with the same name")
print()
ux.error("THIS ACTION CANNOT BE UNDONE! NO ARCHIVE WILL BE CREATED!")
print()

if not ux.yes_no("Are you sure you want to proceed?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

# Double confirmation
if not ux.yes_no(f"Final confirmation: PERMANENTLY DELETE and replace '{cycle_name}'?"):
    ux.info("Operation cancelled by user")
    raise Exception("User cancelled operation")

ux.success("Confirmation received. Proceeding with replacement...")

## Step 1: Delete Current Cycle Directory

In [ ]:
ux.header("Step 1: Deleting Current Cycle Directory")

try:
    # Delete the Active directory
    active_dir = constants.WORKFLOWS_PATH / f"Active_{cycle_name}"
    
    if active_dir.exists():
        shutil.rmtree(active_dir)
        ux.success(f"Deleted directory: {active_dir.name}")
    else:
        ux.warning(f"Active directory not found: {active_dir.name}")
    
    ux.success("✓ Directory deletion complete")
    
except Exception as e:
    ux.error(f"Failed to delete cycle directory: {str(e)}")
    raise

## Step 2: Delete Cycle from Database

In [ ]:
ux.header("Step 2: Deleting Database Records")

try:
    # This will CASCADE delete all related records:
    # - irp_configuration (configurations)
    # - irp_stage (stages)
    # - irp_step (steps)
    # - irp_step_run (step execution history)
    # - irp_batch (batches)
    # - irp_job (jobs)
    success = cycle.delete_cycle(active_cycle['id'])
    
    if success:
        ux.success("Cycle deleted from database (CASCADE deleted all related records)")
    else:
        raise Exception("Failed to delete cycle from database")
    
    ux.success("✓ Database cleanup complete")
    
except Exception as e:
    ux.error(f"Failed to delete cycle: {str(e)}")
    raise

## Step 3: Create Fresh Cycle from Template

In [ ]:
ux.header(f"Step 3: Creating Fresh Cycle: {cycle_name}")

try:
    # Create new cycle with same name
    # This will:
    # 1. Copy template directory to Active_{cycle_name}
    # 2. Create cycle record in database
    # 3. Parse and register stages/steps from template
    success = cycle.create_cycle(cycle_name)
    
    if not success:
        raise Exception("Failed to create new cycle")
    
    ux.success("✓ Fresh cycle created")
    
except Exception as e:
    ux.error(f"Failed to create cycle: {str(e)}")
    raise

## Verification

In [ ]:
ux.header("✓ Verification")

# Verify database
new_active_cycle = cycle.get_active_cycle()
if new_active_cycle and new_active_cycle['cycle_name'] == cycle_name:
    ux.success(f"Active cycle in database: {new_active_cycle['cycle_name']}")
    ux.info(f"Created: {new_active_cycle['created_ts']}")
else:
    ux.error("Verification failed: Active cycle not found or name mismatch")

# Verify filesystem
active_dir = constants.WORKFLOWS_PATH / f"Active_{cycle_name}"
if active_dir.exists():
    ux.success(f"Active directory exists: {active_dir.name}")
else:
    ux.error("Verification failed: Active directory not found")

# Show new cycle progress
new_progress_df = cycle.get_cycle_progress(cycle_name)
if not new_progress_df.empty:
    ux.subheader(f"Registered Steps ({len(new_progress_df)} total)")
    ux.step_progress(new_progress_df)
    ux.success(f"All {len(new_progress_df)} steps are ready to execute")
else:
    ux.warning("No steps found - template may be empty")

print()
ux.header("✓ REPLACEMENT COMPLETE")
ux.success(f"Cycle '{cycle_name}' has been replaced with the latest template")
print()
print("Next steps:")
print(f"  1. Navigate to: Active_{cycle_name}/notebooks/Stage_01_Setup/")
print(f"  2. Start executing step notebooks from the beginning")

## Show Directory Structure

In [ ]:
ux.subheader("Directory Structure")

active_dir = constants.WORKFLOWS_PATH / f"Active_{cycle_name}"

if active_dir.exists():
    print(f"📁 {active_dir.name}/\n")
    
    for item in sorted(active_dir.iterdir()):
        if item.is_dir():
            print(f"  📂 {item.name}/")
            if item.name == 'notebooks':
                for subitem in sorted(item.iterdir()):
                    if subitem.is_dir():
                        print(f"      📂 {subitem.name}/")
                        # Show first 3 notebooks in each stage
                        notebooks = sorted([f for f in subitem.iterdir() if f.suffix == '.ipynb'])
                        for nb in notebooks[:3]:
                            print(f"          📓 {nb.name}")
                        if len(notebooks) > 3:
                            print(f"          ... and {len(notebooks) - 3} more")
else:
    ux.error(f"Directory not found: {active_dir}")

---